In [128]:
from transformers import GPT2LMHeadModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
model = GPT2LMHeadModel.from_pretrained("openai-community/gpt2")

text = "Hello, my dog is cute"
inputs = tokenizer(text, return_tensors="pt")
outputs = model(**inputs, labels=inputs["input_ids"])
for index, logits in enumerate(outputs.logits[0]):
    print(
        f'{tokenizer.decode(inputs["input_ids"][0][index])}: {tokenizer.decode(logits.argmax(dim=-1))}')

Hello: ,
,:  I
 my:  name
 dog:  is
 is:  a
 cute: .


In [55]:
inputs

{'input_ids': tensor([[15496,    11,   616,  3290,   318, 13779]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [18]:
outputs.logits.shape

torch.Size([1, 3, 50257])

In [118]:
import torch
import torch.nn.functional as F
import math

# 单词预测 loss 计算
logits = outputs.logits[0]
total_loss = 0.0
ignore_index = -100
criterion = torch.nn.CrossEntropyLoss()
shift_label = torch.roll(inputs["input_ids"][0], shifts=-1)
shift_label[-1] = ignore_index
count = 0

for seq_index, token_index in enumerate(inputs["input_ids"][0].tolist()):
    current_logits = logits[seq_index]
    # if shift_label[seq_index] != ignore_index:
    loss = criterion(current_logits, shift_label[seq_index])
    total_loss += loss.item()
    count += 1


total_loss/count, criterion(logits, shift_label).item()

(nan, 3.99018931388855)

In [140]:
from cProfile import label
import torch

logits = torch.tensor([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9],
    [10, 11, 12]
], dtype=torch.float32)

labels = torch.tensor([0, 1, 1, -1])

criterion = torch.nn.CrossEntropyLoss(ignore_index=-1)
criterion(logits, labels)

tensor(1.7409)